In [5]:
import json

with open("vk_posts.json", "r") as f:
    data = json.load(f)

In [7]:
test_data = data[0]

In [28]:
import typing as tp
import datetime as dt

class Post(tp.TypedDict):
    id: int
    date: dt.datetime
    text: str
    likes: int
    image: str


def preprocess(group_id:int,payload: dict[str, tp.Any]):
    # get date
    date = dt.datetime.fromtimestamp(payload["date"])
    # get text
    text = payload["text"]
    # get image
    image_urls = []
    for attachment in payload.get("attachments", []):
        if attachment["type"] == "photo":

            image_urls.append(attachment["photo"]["sizes"][-1]["url"])
    return {
        "id": payload["id"],
        "url": f"https://vk.com/wall-{group_id}_{payload['id']}",
        "date": date.isoformat(),
        "text": text,
        "image": image_urls,
    }

In [21]:
print(data[0])

{'id': 6237, 'date': 1708609531, 'type': 'post', 'inner_type': 'wall_wallpost', 'text': '[id3700403|Ольга Чуб]\n\nСанкт-Петербург \n\nСрочно нужен донор крови для собаки. Прооперирована после заворотка желудка. Сейчас падает альбумин и гематокрит.\nКровь DEA 1-\n\n1. г.Санкт Петербург \n2. Вес собаки 22 кг., ВЕО\n3. Необходимый объем 400-500 мл, вес донора от 30-35кг \n4. Резус -\n5. Клиника Прайд, Ленинский пр., 98\n6. Диагноз : После операции падает альбумин и гематокрит\n7. Анализы крови - есть свежие\n8. Пациент: ВЕО Дина\n10. Животное имеет владельца (Игнатьева Галина)\n\nПомогу привезти, отвезти. Если нужно оплатим расходы. \nМой номер +7(921)960-98-33 Ольга (помогаю информационно)\nНомер хозяйки +7(905)222-01-12 Галина\nСсылка на мою страницу https://vk.com/olga_chub\n\n_________________________________________\n#Спб\xa0#нужендонор_pet_donor #нужендонорсобаке\xa0 #НуженДонорсобаке@pet_donor \nБаза данных животных-доноров | Животные-доноры \nЕсли вы готовы помочь и ваши питомцы м

In [ ]:
https://vk.com/wall-135775913_150527

In [29]:
prep = preprocess(5143546, data[0])
prep

{'id': 6237,
 'url': 'https://vk.com/wall-5143546_6237',
 'date': '2024-02-22T16:45:31',
 'text': '[id3700403|Ольга Чуб]\n\nСанкт-Петербург \n\nСрочно нужен донор крови для собаки. Прооперирована после заворотка желудка. Сейчас падает альбумин и гематокрит.\nКровь DEA 1-\n\n1. г.Санкт Петербург \n2. Вес собаки 22 кг., ВЕО\n3. Необходимый объем 400-500 мл, вес донора от 30-35кг \n4. Резус -\n5. Клиника Прайд, Ленинский пр., 98\n6. Диагноз : После операции падает альбумин и гематокрит\n7. Анализы крови - есть свежие\n8. Пациент: ВЕО Дина\n10. Животное имеет владельца (Игнатьева Галина)\n\nПомогу привезти, отвезти. Если нужно оплатим расходы. \nМой номер +7(921)960-98-33 Ольга (помогаю информационно)\nНомер хозяйки +7(905)222-01-12 Галина\nСсылка на мою страницу https://vk.com/olga_chub\n\n_________________________________________\n#Спб\xa0#нужендонор_pet_donor #нужендонорсобаке\xa0 #НуженДонорсобаке@pet_donor \nБаза данных животных-доноров | Животные-доноры \nЕсли вы готовы помочь и ваши

In [31]:
group_id = 5143546
processed = [preprocess(group_id, post) for post in data]

with open(f"vk_posts_preprocessed_{group_id}.json", "w") as f:
    json.dump(processed, f, ensure_ascii=False, indent=4)

In [67]:
from langchain.llms import Ollama

ollama = Ollama(base_url="http://192.168.1.70:11434", model="llama2")

prompt = "language: Russian question: Cократи текст до 30 слов, не выводи, что сократил текст и выдели тип животного в тексте, это может быть либо кошка, либо собака, вот текст:  "

In [70]:

for post in processed[100:200 + 1]:
    text = post["text"]
    summary = ollama(prompt + text)
    with open(f"posts/{post['id']}.json", "w") as f:
        try:

                data = {
                "id": post["id"],
                "link": post["url"],
                "text": text,
                "summary": summary.split(":")[1],
                "image": post["image"],
                "date": post["date"],
            }
        except Exception as e:
            print(e)
            print(summary)
            print(post["id"])
            print(post["text"])
            print(post["image"])
            print(post["url"])
            print(post["date"])
            print("----")
            continue
        json.dump(data, f, ensure_ascii=False, indent=4)

list index out of range
Санкт-Петербург ищем донора крови. Необходимо 130 мл цельной крови группы А. Наша кошка ХПН имеет анемию, поэтому переливание крови планируется в клинике Два сердца. Надеюсь, что донор найдет! Все расходы берём на себя. Марина +79214022595, Дмитрий +79112083428. #Спб #нужендонор_pet_donor #нужендоноркошке #НуженДоноркошке@pet_donor
6121
[id1461732|Лора Калинина]

Санкт-Петербург 

Ищем донора крови. 
Необходимо 130 мл цельной крови группы А.
У нашей кошки ХПН, на фоне которой развилась анемия. 
Переливание крови планируется в клинике Два сердца, по адресу: Львовская 10. В банках пусто, поэтому надеюсь, что донор найдётся! 
Все расходы берём на себя

Марина +79214022595
Дмитрий +79112083428

_________________________________________
#Спб #нужендонор_pet_donor #нужендоноркошке  #НуженДоноркошке@pet_donor 
База данных животных-доноров | Животные-доноры 
Если вы готовы помочь и ваши питомцы могут стать БЕЗВОЗМЕЗДНЫМ донором, внесите свои данные и данные своего питом

KeyboardInterrupt: 

In [60]:
responses[1]

'Текст можно суммировать в 30 слов: "Нужен донор. Важность помощи!"\nТип животного в тексте - кошка (кличка Киса).'